# MVP validation workflow for the Oregon SQM Dashboard project.

Steps:
1. Load raw data tables and display basic info
2. Perform geocoding for all sites & save results
4. Visualize geocoded sites on a Folium map
5. Show bar-charts and scatter-plots.
6. Smoke-test Dash, Flask, and (lightweight) Streamlit app imports
7. Summarize results

## Load raw data tables and display basic info

In [1]:
# Import necessary libraries
from pathlib import Path
import json
import pandas as pd
import time
import importlib
import sys
import logging

In [2]:
# Ensure project root is on path (one level up from 'development')
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Also add the shared directory so that `utils` package can be resolved (matches streamlit_app logic)
SHARED_DIR = PROJECT_ROOT / 'shared'
if str(SHARED_DIR) not in sys.path:
    sys.path.insert(0, str(SHARED_DIR))

RAW_DIR = PROJECT_ROOT / 'shared' / 'data' / 'raw'
PROCESSED_DIR = PROJECT_ROOT / 'shared' / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Shared dir added to sys.path: {SHARED_DIR}")
print(f"Raw data dir: {RAW_DIR}")
print(f"Processed data dir: {PROCESSED_DIR}")

# List raw CSVs
for p in RAW_DIR.glob('*.csv'):
    print('Found raw file:', p.name)

Project root: /home/vidit-agrawal/projects/darksky-oregon-dashboard
Shared dir added to sys.path: /home/vidit-agrawal/projects/darksky-oregon-dashboard/shared
Raw data dir: /home/vidit-agrawal/projects/darksky-oregon-dashboard/shared/data/raw
Processed data dir: /home/vidit-agrawal/projects/darksky-oregon-dashboard/shared/data/processed
Found raw file: cloudy_night_measurements.csv
Found raw file: sites_locations.csv
Found raw file: cloud_coverage.csv
Found raw file: sites_coordinates.csv
Found raw file: clear_night_measurements.csv
Found raw file: longterm_trends.csv
Found raw file: milky_way_visibility.csv


In [3]:
from shared.utils.data_processing import OregonSQMProcessor

In [4]:
process_data = OregonSQMProcessor(
    data_dir=PROJECT_ROOT / 'shared' / 'data'
)
raw_dfs = process_data.load_raw_data()

INFO:shared.utils.data_processing:Initializing OregonSQMProcessor with data directory: /home/vidit-agrawal/projects/darksky-oregon-dashboard/shared/data
INFO:shared.utils.data_processing:Loaded sites: 63 records
INFO:shared.utils.data_processing:Loaded geocode: 63 records
INFO:shared.utils.data_processing:Loaded clear_measurements: 58 records
INFO:shared.utils.data_processing:Loaded cloudy_measurements: 60 records
INFO:shared.utils.data_processing:Loaded trends: 32 records
INFO:shared.utils.data_processing:Loaded milky_way: 57 records
INFO:shared.utils.data_processing:Loaded cloud_coverage: 60 records


In [5]:
raw_dfs.keys()

dict_keys(['sites', 'geocode', 'clear_measurements', 'cloudy_measurements', 'trends', 'milky_way', 'cloud_coverage'])

In [6]:
for key in raw_dfs:
    print(key)
    display(raw_dfs[key].info())

sites
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            63 non-null     object 
 1   Install Number  63 non-null     int64  
 2    Address        0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ KB


None

geocode
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   site_name  63 non-null     object 
 1   latitude   63 non-null     float64
 2   longitude  63 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.6+ KB


None

clear_measurements
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   site_name                          58 non-null     object 
 1   median_brightness_mag_arcsec2      58 non-null     float64
 2   bortle_sky_level                   58 non-null     int64  
 3   median_linear_scale_flux_ratio     58 non-null     float64
 4   x_brighter_than_darkest_night_sky  58 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.4+ KB


...

None

milky_way
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   site_name                     57 non-null     object 
 1   difference_index_mag_arcsec2  57 non-null     float64
 2   ratio_index                   57 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.5+ KB


None

cloud_coverage
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   site_name                               60 non-null     object 
 1   percent_clear_night_samples_all_months  60 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


None

## Perform geocoding for all sites & save results

In [7]:
# # Geocode all sites using only Google Maps API
# import requests
# import pandas as pd

# # API_KEY = '<redacted for privacy reasons>'  # <-- Replace with your Google Maps API key

# def google_maps_geocode(address, api_key):
#     url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
#     resp = requests.get(url)
#     if resp.status_code == 200:
#         results = resp.json().get('results', [])
#         if results:
#             loc = results[0]['geometry']['location']
#             return loc['lat'], loc['lng']
#     return None, None

# site_names = raw_dfs['sites']['Name'].dropna().tolist()

# results = []
# for i, site_name in enumerate(site_names, 1):
#     address = f"{site_name}, Oregon, USA"
#     print(f"[{i}/{len(site_names)}] Geocoding: {address}")
#     lat, lon = google_maps_geocode(address, API_KEY)
#     results.append({"site_name": site_name, "latitude": lat, "longitude": lon})


# geocode_df = pd.DataFrame(results)
# geocode_df.to_csv(PROJECT_ROOT / 'shared/data/geospatial/sites_coordinates.csv', index=False)
# geocode_df.head()

In [8]:
# geocode_df.to_csv(PROJECT_ROOT / 'shared/data/geospatial/sites_coordinates.csv', index=False)

## Visualize geocoded sites on a Folium map

In [9]:
geocode_df = pd.read_csv(PROJECT_ROOT / 'shared/data/raw/sites_coordinates.csv')

In [10]:
# # 4. Map visualization (requires successful geocoding)
# import folium

# fmap = folium.Map(location=[43.9,-120.6], zoom_start=6, tiles='CartoDB positron')
# for i, row in geocode_df.dropna(subset=['latitude', 'longitude']).iterrows():
#     folium.Marker(
#         location=[row['latitude'], row['longitude']],
#         popup=row['site_name'],
#         icon=folium.Icon(color='blue', icon='info-sign')
#     ).add_to(fmap)

#display(fmap)
    

## Visualization

In [11]:
# from shared.utils.visualizations import create_ranking_chart

# create_ranking_chart(
#     sites_df=raw_dfs['clear_measurements'],
#     y_col='median_brightness_mag_arcsec2'
# )

In [12]:
from shared.utils.visualizations import create_interactive_2d_plot

create_interactive_2d_plot(
    df=raw_dfs['clear_measurements'],
    x_col='median_brightness_mag_arcsec2',
    y_col='median_linear_scale_flux_ratio',
    hover_cols=['bortle_sky_level', 'x_brighter_than_darkest_night_sky'],
)